## Install depedencies

In [ ]:
# Ignore any "Restart session" suggestions
!pip install rasa[spacy]
!python  -m spacy download en_core_web_md
!pip install nest-asyncio

## Import the generated model

---

Drag and drop the generated `tar.gz` file in the Files on your left and change the `filename` variable to the tar.gz name and run the code to untar the files and change the current working directory.

In [ ]:
import os
import tarfile
import shutil

filename = 'codegen-9184ced9.tar.gz'  # Replace with the actual filename
os.chdir('/content')

# Step 1: Ensure the file exists
if os.path.exists(filename):
    # Step 2: Untar the file
    if filename.endswith(".tar.gz"):
        with tarfile.open(filename, "r:gz") as tar:
            main_dir = tar.getnames()[0].split('/')[0]

            # Check if the directory already exists and delete it
            if os.path.exists(main_dir):
                shutil.rmtree(main_dir)

            # Extract the tar file
            tar.extractall()

        # Step 3: Change the working directory to the main directory
        os.chdir(main_dir)
        print(f"Current working directory: {os.getcwd()}")
    else:
        print("Please specify a valid .tar.gz file")
else:
    print("File not found. Please ensure the filename is correct and the file is uploaded.")

## Train the Virtual Assistant model

For each different model, delete all old Rasa files and drag-and-drop the new ones.

In [ ]:
# Train the Virtual Assistant model
!rasa train

## Deploy the assistant

In [ ]:
# Setup the environment
import nest_asyncio
import threading
import os
import rasa

nest_asyncio.apply()

# Function to run Rasa server
def run_rasa():
    os.system('rasa run -m models --enable-api --cors "*" --debug')

# Function to run Rasa action server
def run_action_server():
    os.system('rasa run actions --debug')

# Create threads for Rasa server and action server
rasa_thread = threading.Thread(target=run_rasa)
action_thread = threading.Thread(target=run_action_server)

# Start the threads
rasa_thread.start()
action_thread.start()


## Chat with your model

In [ ]:
import requests

# Interactive loop to prompt user for input and print Rasa responses
def interactive_chat():
    rasa_url = "http://localhost:5005/webhooks/rest/webhook"
    while True:
        message = input("You: ")
        if message.lower() == "exit":
            print("Exiting chat...")
            break
        payload = {"sender": "user", "message": message}
        response = requests.post(rasa_url, json=payload)
        for res in response.json():
            print(f"Bot: {res['text']}")

# Start the interactive chat and chat with your model
interactive_chat()
